# 13. Uso de servicios web

Alejandro E. Martínez Castro (amcastro@ugr.es)

Tomado de C. Severance. "Python for Informatics"

## Formatos de ficheros para intercambio de datos en la web

Existen dos formatos básicos que se emplean en el intercambio de datos en la web. Por un lado, el [XML](https://es.wikipedia.org/wiki/Extensible_Markup_Language) (eXtensive Markup Languaje) y el [JSON](https://es.wikipedia.org/wiki/JSON) (JavaScript Object Notation). 

El primero de ellos, XML, se emplea fundamentalmente para el intercambio de datos en formato documento. 

El segundo de ellos, JSON, se emplea para intercambiar diccionarios, listas, etc. 

En lo que sigue veremos cómo se estructuran ambos. 

## 1. eXtensive Markup Languaje - XML

Un fichero XML tiene un aspecto muy parecido a uno en HTML, pero XML es más estructurado. Veamos un ejemplo de XML: 

    <persona>
        <nombre>Alejandro</nombre>
        <teléfono type="intl">
            +34 958 11 22 33
        </teléfono>
        <email oculto="yes" />
    </persona>

Cada pareja de etiquetas de apertura (como `<persona>`) y cierre (como `<\persona>`) representa un *elemento* o *nodo* con el mismo nombre que la etiqueta (como `persona`). Cada elemento puede tener texto, atributos (e.g. **oculto**), y otros elementos anidados. Si un elemento XML está vacío (es decir, no tiene contenido), debe ser delimitado por una etiqueta especial "semi-cerrada", como `<email \>`.

Es útil pensar en un documento XML como una estructura arborescente, en la cual existe un elemento en el nivel superior (en este caso `persona`) y otras etiquetas aparecen como hijos (como `phone`)

### 1.1 Análisis del código XML

En el siguiente bloque se muestra un ejemplo de extracción de datos a partir de un fichero XML, de forma simple. 

In [1]:
import xml.etree.ElementTree as ET

data = '''
<persona>
  <nombre>Alejandro</nombre>
  <teléfono type="intl">
    +34 958 11 22 33
  </teléfono>
  <email oculto="sí" />
</persona>'''

arbol = ET.fromstring(data)
print('Nombre:', arbol.find('nombre').text)
print('Atributo de email: ¿Es oculto?:', arbol.find('email').get('oculto'))
print('Teléfono:', arbol.find('teléfono').text)

Nombre: Alejandro
Atributo de email: ¿Es oculto?: sí
Teléfono: 
    +34 958 11 22 33
  


En el bloque anterior, obsérvese que la triple comilla ('''), que también puede ponerse como ("""), permite generar cadenas de texto de multiples líneas. 

La llamada a `fromstrings` convierte la cadena de texto XML en un "árbol" de elementos XML. Cuando el XML está en un árbol, es posible, mediante ciertos métodos, acceder a la información de forma ordenada. La función `find` busca en el fichero XML y devuelve el elemento que coincide con el criterio. Observando este ejemplo, `tree.find` obtiene el contenido de la etiqueta `<nombre>`, o el atributo de `<email>`.

El uso de un analizador como `ElementTree` permite extraer datos de un XML sin procuparnos de las reglas de la sintaxis XML. 

### 1.2. Bucle entre nodos

Normalmente un XML tiene muchos nodos, y es necesario escribir un bucle para procesar todos los nodos. En el siguiente ejemplo, iteramos entre todos los nodos de `<usuario>`.

In [2]:
import xml.etree.ElementTree as ET

entrada = '''
<plantilla>
  <usuarios>
    <usuario x="2">
      <id>001</id>
      <nombre>Alejandro</nombre>
    </usuario>
    <usuario x="7">
      <id>009</id>
      <nombre>Pedro</nombre>
    </usuario>
  </usuarios>
</plantilla>'''

trabajadores = ET.fromstring(entrada)
lista = trabajadores.findall('usuarios/usuario') # Observe cómo selecciona "usuario" dentro de "usuarios"
print('Contador de usuarios:', len(lista))

for item in lista:
    print('Nombre', item.find('nombre').text)
    print('Identificador', item.find('id').text)
    print('Atributo', item.get('x'))

Contador de usuarios: 2
Nombre Alejandro
Identificador 001
Atributo 2
Nombre Pedro
Identificador 009
Atributo 7


Es importante incluir todos los elementos "padre" en la búsqueda con `findall`. De lo contrario, Python no encontrará la información deseada. Observe el siguiente ejemplo: 

In [3]:
import xml.etree.ElementTree as ET

entrada = '''
<plantilla>
  <usuarios>
    <usuario x="2">
      <id>001</id>
      <nombre>Alejandro</nombre>
    </usuario>
    <usuario x="7">
      <id>009</id>
      <nombre>Pedro</nombre>
    </usuario>
  </usuarios>
</plantilla>'''

trabajadores = ET.fromstring(entrada)

# Primer caso: búsqueda de usuario dentro de usuarios
lista = trabajadores.findall('usuarios/usuario') 
print('Contador de usuarios:', len(lista))

# Segundo caso: búsqueda de usuario dentro de plantilla. No hay nivel inmediat inferior denominado "usuario"
lista2 = trabajadores.findall('usuario')
print('Contador de usuarios:', len(lista2))

Contador de usuarios: 2
Contador de usuarios: 0


## 2. JavaScript Object Notation - JSON

El formato JSON fue inspirado en los formatos de array y objeto del lenguaje JavaScript. Pero Python fue inventado antes que JavaScript, y es claro que el formato de diccionarios y listas de Python influyó en la sintaxis de JSON. El formato JSON es una combinación de diccionarios y listas de Python. Observe el siguiente bloque JSON. 

    {
      "nombre" : "Alejandro",
      "teléfono" : {
        "type" : "intl",
        "número" : "+34 958 11 22 33"
        },
      "email" : {
        "oculto" : "sí"
       }
    }
    
En comparación con el XML anterior, se observan diferencias. En un XML, el atributo "intl" se puede añadir a la etiqueta "teléfono". En un JSON la información se estructura en parejas de "palabra-clave: contenido", como en un diccionario de Python. 

En general, la estructura JSON es más simple que la XML, pues JSON tiene menos capacidades que XML. La principal ventaja de JSON es que emplea directamente diccionarios y listas (una combinación de ambos). Y puesto que los lenguajes suelen tener de alguna forma el equivalente a diccionarios y listas de Python, JSON es un formato muy natural para intercambio de datos. 

### 2.1. Análisis de código JSON

El fichero JSON contiene diccionarios y listas. En este ejemplo representamos una lista de usuarios, donde cada usuario incluye una pareja de etiquetas-valores. 

En este ejemplo emplearemos la librería `json` para analizar ficheros JSON y extraer la información que se desee. Es interesante comparar con la representación XML. El JSON tiene menos detalles. Debemos saber de antemano que tendremos una lista, y que esta lista es de usuarios, y además, que cada usuario es una pareja de etiquetas-valores. El JSON es más directo, y menos descriptivo que el XML. 

In [4]:
import json

data = '''
[
  { "id" : "001",
    "x" : "2",
    "nombre" : "Alejandro"
  } ,
  { "id" : "009",
    "x" : "7",
    "nombre" : "Pedro"
  }
]'''

info = json.loads(data)
print('User count:', len(info))

for item in info:
    print('Nombre', item['nombre'])
    print('Id', item['id'])
    print('Atributo', item['x'])

User count: 2
Nombre Alejandro
Id 001
Atributo 2
Nombre Pedro
Id 009
Atributo 7


Si compara con el fichero XML, lo que se obtiene de `json.loads()` es una lista que se recorre con un bucle `for`.

No es necesario "traducir" lo que se obtiene del fichero JSON, pues son listas nativas en código Python. 

En general, existe una tendencia en diseño Web a utilizar ficheros JSON, en lugar de XML. 

## 3. APIs (Application Programming Interfaces)

Las APIs son una forma de utilizar servicios web de determinados programas. Estas APIs las podemos incorporar en nuestros programas. 

Esto es algo que observamos de manera habitual cuando navegamos por la web. Vemos páginas de aerolíneas que ofrecen servicios de hotel. La página conecta con una API del hotel que devuelve datos para poder ser insertados en la web de la aerolínea. 

Es habitual, para usar la API de un distribuidor de servicios web, tener que darse de alta, y tener una contraseña. Estas APIs tienen un uso limitado. Para poder ampliar las herramientas APIs se utiliza el `web scraping`.

### Ejemplo: Servicio de geolocalización de Google: Google Places

En este ejemplo se empleará la API de Google Places. Se pedirán datos de la localización de un emplazamiento, en el servicio de Google. Se obtendrán los datos en un JSON. 

In [ ]:
import urllib.request, urllib.parse, urllib.error
import json
import ssl

api_key = False
# Si dispone de una clave para API de Google Places, introducir aquí
# api_key = 'AIzaSy___IDByT70'
# https://developers.google.com/maps/documentation/geocoding/intro

if api_key is False:
    api_key = 42
    serviceurl = 'http://py4e-data.dr-chuck.net/json?'
else :
    serviceurl = 'https://maps.googleapis.com/maps/api/geocode/json?'

# Ignorar los errores de certificado SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

while True:
    address = input('Introduzca localización: ')
    if len(address) < 1: break

    parms = dict()
    parms['address'] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Accediendo a ', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Recuperados ', len(data), 'caracteres')

    try:
        js = json.loads(data)
    except:
        js = None

    if not js or 'status' not in js or js['status'] != 'OK':
        print('==== Fallo al acceder y recuperar ====')
        print(data)
        continue

    print(json.dumps(js, indent=4))

    lat = js['results'][0]['geometry']['location']['lat']
    lng = js['results'][0]['geometry']['location']['lng']
    print('lat', lat, 'lng', lng)
    location = js['results'][0]['formatted_address']
    print(location)

Introduzca localización: Granada
Accediendo a  http://py4e-data.dr-chuck.net/json?address=Granada&key=42
Recuperados  1702 caracteres
{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Granada",
                    "short_name": "Granada",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Granada",
                    "short_name": "Granada",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "Andalusia",
                    "short_name": "AL",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
             